# Used car price prediction

### Objective

Build a linear regression model to predict the prices of used cars.

### Data Description
- Brand: brand name of the car
- Model Name: model name of the car
- Location: Location in which the car is being sold or is available for purchase (cities)
- Year: Manufacturing year of the car
- Kilometers_driven: The total kilometers driven in the car by the previous owner(s) in km
- Fuel_Type: The type of fuel used by the car (Petrol, Diesel, Electric, CNG, LPG)
- Transmission: The type of transmission used by the car (Automatic/Manual)
- Owner_Type: Type of ownership
- Mileage: The standard mileage offered by the car company in kmpl or km/kg
- Engine: The displacement volume of the engine in CC
- Power: The maximum power of the engine in bhp
- Seats: The number of seats in the car
- New_Price: The price of a new car of the same model in INR Lakhs (1 Lakh = 100,000 INR)
- Price: The price of the used car in INR Lakhs

The used car dataset is taken from https://www.kaggle.com/

## Importing  libraries


In [ ]:
# Libraries to manipulate the data
import numpy as np
import pandas as pd

# import libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# set default parameters
sns.set()

# split the data between training and test
from sklearn.model_selection import train_test_split

# import linear regression
from sklearn.linear_model import LinearRegression

# import metrics
from sklearn.metrics import mean_absolute_error , mean_squared_error, r2_score

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

## Loading the dataset

In [ ]:
# to read data from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cars = pd.read_csv("/content/drive/MyDrive/MLPractice/used_cars_data.csv")

## Data Overview

### Showing the first few rows of the dataset

In [ ]:
# create copy of the data to avoid any changes in original data.
data = cars.copy()

In [ ]:
data.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,New_Price,Price,Mileage,Engine,Power,Brand,Model
0,Mumbai,2010,72000.0,CNG,Manual,First,5.0,5.51,1.75,26.60,998.0,58.16,maruti,wagon
1,Pune,2015,41000.0,Diesel,Manual,First,5.0,16.06,12.50,19.67,1582.0,126.20,hyundai,creta
2,Chennai,2011,46000.0,Petrol,Manual,First,5.0,8.61,4.50,18.20,1199.0,88.70,honda,jazz
3,Chennai,2012,87000.0,Diesel,Manual,First,7.0,11.27,6.00,20.77,1248.0,88.76,maruti,ertiga
4,Coimbatore,2013,40670.0,Diesel,Automatic,Second,5.0,53.14,17.74,15.20,1968.0,140.80,audi,a4


### Checking the number of rows and columns

In [ ]:
data.shape

(7252, 14)

*   There are 7252 rows and 14 columns




In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7252 entries, 0 to 7251
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           7252 non-null   object 
 1   Year               7252 non-null   int64  
 2   Kilometers_Driven  7251 non-null   float64
 3   Fuel_Type          7252 non-null   object 
 4   Transmission       7252 non-null   object 
 5   Owner_Type         7252 non-null   object 
 6   Seats              7199 non-null   float64
 7   New_Price          7252 non-null   float64
 8   Price              6019 non-null   float64
 9   Mileage            7169 non-null   float64
 10  Engine             7206 non-null   float64
 11  Power              7077 non-null   float64
 12  Brand              7252 non-null   object 
 13  Model              7252 non-null   object 
dtypes: float64(7), int64(1), object(6)
memory usage: 793.3+ KB


*  There are 6 objects type or category columns and 7 numerical type columns




In [ ]:
data.describe()

,Year,Kilometers_Driven,Seats,New_Price,Price,Mileage,Engine,Power
count,7252.000000,7251.000000,7199.000000,7252.000000,6019.000000,7169.000000,7206.000000,7077.000000
mean,2013.365830,57811.654255,5.280456,21.308387,9.479468,18.346715,1616.590064,112.764474
std,3.254405,37502.061260,0.809327,24.257816,11.187917,4.158170,595.324779,53.497297
min,1996.000000,171.000000,2.000000,3.910000,0.440000,6.400000,72.000000,34.200000
25%,2011.000000,34000.000000,5.000000,7.880000,3.500000,15.300000,1198.000000,75.000000
50%,2014.000000,53416.000000,5.000000,11.300000,5.640000,18.200000,1493.000000,94.000000
75%,2016.000000,73000.000000,5.000000,21.697500,9.950000,21.100000,1968.000000,138.100000
max,2019.000000,775000.000000,10.000000,375.000000,160.000000,33.540000,5998.000000,616.000000




*   Mean and median are almost same for Mileage and Seats so Mileage and Seats have symmetric distribution.




## Data preprocessing

### Checking for duplicate values

In [ ]:
# checking for duplicate values
data.duplicated().sum()

2



*   2 duplicate values in the data
*   Let's find out the duplicate values



In [ ]:
data[data.duplicated(keep=False)== True]

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,New_Price,Price,Mileage,Engine,Power,Brand,Model
3623,Hyderabad,2007,52195.0,Petrol,Manual,First,5.0,4.36,1.75,19.7,796.0,46.3,maruti,alto
4781,Hyderabad,2007,52195.0,Petrol,Manual,First,5.0,4.36,1.75,19.7,796.0,46.3,maruti,alto
6940,Kolkata,2017,13000.0,Diesel,Manual,First,5.0,13.58,NaN,26.0,1498.0,98.6,honda,city
7077,Kolkata,2017,13000.0,Diesel,Manual,First,5.0,13.58,NaN,26.0,1498.0,98.6,honda,city




*   Rows 3623 and 4781 are same
*   Rows 6940 and 7077 are same
*   We can remove duplicate rows

In [ ]:
# remove row 4781
data.drop(4781, inplace=True)

# remvoe row 7077
data.drop(7077, inplace=True)

In [ ]:
# checking for duplicate values
data.duplicated().sum()

0

*  Now there are no duplicate values in dataset.




## Missing Values Treatment

In [ ]:
data.isnull().sum()

,0
Location,0
Year,0
Kilometers_Driven,1
Fuel_Type,0
Transmission,0
Owner_Type,0
Seats,53
New_Price,0
Price,1232
Mileage,83




*   There are missing values in columns Kilometers_Driven, Seats, Price, Mileage, Engine and Power.



In [ ]:
# as price is the target variable so use the data where price is not missing.
data = data[data["Price"].notna()].copy()

# check if missing data for price is removed from dataset
data.isnull().sum()

,0
Location,0
Year,0
Kilometers_Driven,1
Fuel_Type,0
Transmission,0
Owner_Type,0
Seats,42
New_Price,0
Price,0
Mileage,70




*   Filling missing values with median value of feature by brand and model




In [ ]:
cols_list = ["Kilometers_Driven","Seats", "Mileage", "Engine", "Power"]

for col in cols_list:
    data[col] = data.groupby(["Brand", "Model"])[col].transform(
        lambda x: x.fillna(x.median())
    )

data.isnull().sum()

,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Seats,3
New_Price,0
Price,0
Mileage,9


*   Filling missing values with median value of feature by brand




In [ ]:
cols_list = ["Seats","Mileage", "Engine", "Power"]

for col in cols_list:
    data[col] = data.groupby(["Brand"])[col].transform(
        lambda x: x.fillna(x.median())
    )

data.isnull().sum()

,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Seats,0
New_Price,0
Price,0
Mileage,1



*   Filling missing values with median value of feature




In [ ]:
cols_list = ["Mileage", "Power"]

for col in cols_list:
    data[col] = data[col].fillna(data[col].median())

data.isnull().sum()

,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Seats,0
New_Price,0
Price,0
Mileage,0


## Model Building - Linear Regression


###  Categorical columns Treatement


In [ ]:
object_columns = data.select_dtypes(include='object')
unique_counts = object_columns.nunique()
print(unique_counts)

Location         11
Fuel_Type         5
Transmission      2
Owner_Type        4
Brand            30
Model           211
dtype: int64




*   There are 211 category in Model column and its not a small category and If we use dummy variabeks then it will increase the number of features in the model.
*   There are 30 different values in Brand
*  As Model will increase features in the model so we will create 2 different models
    1.   Model with all columns
    2.   Model without Model variable








In [ ]:
# for adjusted R-squared
def adjusted_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))

# check performance of a regression model
def model_performance_regression(model, predictors, target):
    predictions = model.predict(predictors)
    r2 = r2_score(target, predictions)  #  R-squared
    adjr2 = adjusted_r2_score(predictors, target, predictions)  #  adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, predictions))  #  RMSE
    mae = mean_absolute_error(target, predictions)  #  MAE

    # creating a dataframe of metrics
    data_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adjusted R-squared": adjr2,
        },
        index=[0],
    )
    print(data_perf)
    return data_perf

### Model with all variables

In [ ]:
X = data.drop(["Price"], axis=1) # remove target variable
y = data["Price"]

In [ ]:
# creating dummy variables
X = pd.get_dummies(
    X,
    columns=X.select_dtypes(include=["object", "category"]).columns.tolist(),
    drop_first=True,
)

X.head()

,Year,Kilometers_Driven,Seats,New_Price,Mileage,Engine,Power,Location_Bangalore,Location_Chennai,Location_Coimbatore,...,Model_xenon,Model_xf,Model_xj,Model_xuv300,Model_xuv500,Model_xylo,Model_yeti,Model_z4,Model_zen,Model_zest
0,2010,72000.0,5.0,5.51,26.60,998.0,58.16,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2015,41000.0,5.0,16.06,19.67,1582.0,126.20,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2011,46000.0,5.0,8.61,18.20,1199.0,88.70,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,2012,87000.0,7.0,11.27,20.77,1248.0,88.76,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,2013,40670.0,5.0,53.14,15.20,1968.0,140.80,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# splitting the data in 70:30 ratio for train to test data

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# fitting a linear model
linear_reg_model1 = LinearRegression()
linear_reg_model1.fit(x_train, y_train)

LinearRegression()

In [ ]:
# Checking model performance on train set
print("Training Performance:")
linear_reg_model1_perf_train = model_performance_regression(
    linear_reg_model1, x_train, y_train
)
linear_reg_model1_perf_train

Training Performance:
       RMSE      MAE  R-squared  Adjusted R-squared
0  4.177602  2.24435   0.865733            0.856752


,RMSE,MAE,R-squared,Adjusted R-squared
0,4.177602,2.24435,0.865733,0.856752


In [ ]:
# Checking model performance on test set
print("Test Performance:")
linear_reg_model1_perf_test = model_performance_regression(linear_reg_model1, x_test, y_test)
linear_reg_model1_perf_test

Test Performance:


,RMSE,MAE,R-squared,Adjusted R-squared
0,4.306052,2.387067,0.837175,0.809281


### Model without dummy variables for variable Model

In [ ]:
# defining the dependent and independent variables
X = data.drop(["Price", "Model"], axis=1)
y = data["Price"]

# creating dummy variables
X = pd.get_dummies(
    X,
    columns=X.select_dtypes(include=["object", "category"]).columns.tolist(),
    drop_first=True,
)

# splitting the data in 70:30 ratio for train to test data

x_train2, x_test2, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1
)

In [ ]:
# fitting a linear model
linear_reg_model2 = LinearRegression()
linear_reg_model2.fit(x_train2, y_train)

LinearRegression()

In [ ]:
# Checking model performance on train set
print("Training Performance:")
linear_reg_model2_perf_train = model_performance_regression(
    linear_reg_model2, x_train2, y_train
)
linear_reg_model2_perf_train

Training Performance:
       RMSE       MAE  R-squared  Adjusted R-squared
0  5.373599  2.885305    0.77785            0.774964


,RMSE,MAE,R-squared,Adjusted R-squared
0,5.373599,2.885305,0.77785,0.774964


In [ ]:
# Checking model performance on test set
print("Test Performance:")
linear_reg_model2_perf_test = model_performance_regression(linear_reg_model2, x_test2, y_test)
linear_reg_model2_perf_test

Test Performance:


,RMSE,MAE,R-squared,Adjusted R-squared
0,4.708758,2.758179,0.805296,0.799292


### Performance Comparison of Models

In [ ]:
models_train_comparison = pd.concat(
    [linear_reg_model1_perf_train.T, linear_reg_model2_perf_train.T,], axis=1,
)

models_train_comparison.columns = [
    "Linear Regression with all variable",
    "Linear Regression without dummy variables for Model",
]

print("Training performance comparison:")
models_train_comparison

Training performance comparison:


,Linear Regression with all variable,Linear Regression without dummy variables for Model
RMSE,4.177602,5.373599
MAE,2.244350,2.885305
R-squared,0.865733,0.777850
Adjusted R-squared,0.856752,0.774964



# Conclusion
*   Linear Regression with all variable Model performace is good as RMSE and MAE are low and R-squared and adjusted R-squared are better than Linear Regression without dummy variables for Model


